In [ ]:
# Cell 1: Install required libraries
!pip install -q transformers torch accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 20.0 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Model name
model_name = "Qwen/Qwen2.5-1.5B-Instruct"  # Using Qwen2.5 as Qwen3-1.7B might not be available

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

print(f"Model loaded successfully on {model.device}")
print(f"Model size: {sum(p.numel() for p in model.parameters()) / 1e9:.2f}B parameters")

Loading tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Model loaded successfully on cuda:0
Model size: 1.54B parameters


In [ ]:
# Cell 3: Create helper function for inference
def generate_response(prompt, max_new_tokens=1024, temperature=0.7):
    """
    Generate a response from the model given a prompt.

    Args:
        prompt: The input prompt string
        max_new_tokens: Maximum number of tokens to generate
        temperature: Sampling temperature (higher = more creative)

    Returns:
        Generated text response
    """
    # Format the prompt as a conversation
    messages = [
        {"role": "user", "content": prompt}
    ]

    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Tokenize
    inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode and extract only the generated part
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove the input prompt from output
    response = full_output.split(prompt)[-1].strip()

    return response

print("Helper function created successfully!")

Helper function created successfully!


In [ ]:
# Cell 4: Test 1 - Instruction Following Capability
print("=" * 80)
print("TEST 1: INSTRUCTION FOLLOWING CAPABILITY")
print("=" * 80)

instruction_tests = [
    {
        "name": "Simple Format Following",
        "prompt": """Please list 3 benefits of exercise. Format your answer as:
Benefit 1: [description]
Benefit 2: [description]
Benefit 3: [description]"""
    },
    {
        "name": "Multi-step Instructions",
        "prompt": """Follow these steps:
1. Define what artificial intelligence is in one sentence
2. Give an example of AI in daily life
3. End with the word 'Complete'"""
    },
    {
        "name": "Constraint Following",
        "prompt": "Write a short story (exactly 3 sentences) about a robot. Each sentence must start with a different letter: R, O, B."
    }
]

for i, test in enumerate(instruction_tests, 1):
    print(f"\n--- Test {i}: {test['name']} ---")
    print(f"Prompt: {test['prompt']}\n")

    response = generate_response(test['prompt'], max_new_tokens=300)
    print(f"Response:\n{response}")
    print("\n" + "-" * 80)

TEST 1: INSTRUCTION FOLLOWING CAPABILITY

--- Test 1: Simple Format Following ---
Prompt: Please list 3 benefits of exercise. Format your answer as:
Benefit 1: [description]
Benefit 2: [description]
Benefit 3: [description]

Response:
assistant
Sure! Here are three benefits of exercise:

Benefit 1: Improves cardiovascular health and reduces the risk of heart disease.
Benefit 2: Increases muscle strength and endurance, leading to better overall physical fitness.
Benefit 3: Enhances mental well-being by reducing stress, anxiety, and depression.

--------------------------------------------------------------------------------

--- Test 2: Multi-step Instructions ---
Prompt: Follow these steps:
1. Define what artificial intelligence is in one sentence
2. Give an example of AI in daily life
3. End with the word 'Complete'

Response:
assistant
Artificial Intelligence (AI) refers to the simulation of human intelligence processes by computer systems, including learning, reasoning, and self-cor

In [ ]:
# Cell 5: Test 2 - Code Writing and Generation Capability
print("=" * 80)
print("TEST 2: CODE WRITING AND GENERATION CAPABILITY")
print("=" * 80)

code_tests = [
    {
        "name": "Simple Function",
        "prompt": "Write a Python function that takes a list of numbers and returns the sum of even numbers only. Include comments."
    },
    {
        "name": "Algorithm Implementation",
        "prompt": "Write a Python function to implement binary search on a sorted list. Include error handling."
    },
    {
        "name": "Data Processing",
        "prompt": "Write Python code to read a CSV file, filter rows where age > 25, and calculate the average salary. Use pandas."
    },
    {
        "name": "Code Debugging",
        "prompt": """Find and fix the bug in this code:
def calculate_average(numbers):
    total = 0
    for num in numbers:
        total += num
    return total / 0

Explain what was wrong and provide the corrected version."""
    }
]

for i, test in enumerate(code_tests, 1):
    print(f"\n--- Test {i}: {test['name']} ---")
    print(f"Prompt: {test['prompt']}\n")

    response = generate_response(test['prompt'], max_new_tokens=512)
    print(f"Response:\n{response}")
    print("\n" + "-" * 80)

TEST 2: CODE WRITING AND GENERATION CAPABILITY

--- Test 1: Simple Function ---
Prompt: Write a Python function that takes a list of numbers and returns the sum of even numbers only. Include comments.

Response:
assistant
Certainly! Below is a Python function that sums up all the even numbers from a given list:

```python
def sum_even_numbers(numbers):
    """
    This function takes a list of integers as input and returns the sum of all even numbers in the list.
    
    :param numbers: List[int] - A list of integers.
    :return: int - The sum of all even numbers in the list.
    """
    total_sum = 0
    
    # Loop through each number in the list
    for num in numbers:
        # Check if the current number is even (num % 2 == 0)
        if num % 2 == 0:
            # Add the even number to the total sum
            total_sum += num
            
    return total_sum

# Example usage:
numbers_list = [1, 2, 3, 4, 5, 6]
result = sum_even_numbers(numbers_list)
print(f"The sum of even n

In [ ]:
# Cell 6: Test 3 - Task Decomposition into Subtasks
print("=" * 80)
print("TEST 3: TASK DECOMPOSITION CAPABILITY")
print("=" * 80)

decomposition_tests = [
    {
        "name": "Project Planning",
        "prompt": "Break down the task of 'Building a simple blog website' into subtasks. List each subtask with a brief description."
    },
    {
        "name": "Problem Solving",
        "prompt": "I need to analyze customer feedback data and create a report. Break this into step-by-step subtasks."
    },
    {
        "name": "Complex Algorithm",
        "prompt": "Decompose the task of 'Implementing a recommendation system' into subtasks. Be specific about each step."
    }
]

for i, test in enumerate(decomposition_tests, 1):
    print(f"\n--- Test {i}: {test['name']} ---")
    print(f"Prompt: {test['prompt']}\n")

    response = generate_response(test['prompt'], max_new_tokens=1024)
    print(f"Response:\n{response}")
    print("\n" + "-" * 80)

TEST 3: TASK DECOMPOSITION CAPABILITY

--- Test 1: Project Planning ---
Prompt: Break down the task of 'Building a simple blog website' into subtasks. List each subtask with a brief description.

Response:
assistant
Certainly! Here's how you can break down the task of "Building a simple blog website" into manageable subtasks:

### 1. **Define Your Blog’s Purpose and Content**
   - **Description**: Decide what type of content your blog will feature (e.g., personal thoughts, news updates, product reviews).
   - **Subtasks**:
     - Determine the frequency of posts.
     - Identify key topics or themes.

### 2. **Choose a Hosting Service**
   - **Description**: Select a platform to host your blog (e.g., WordPress, Blogger, Medium, etc.).
   - **Subtasks**:
     - Research different hosting services.
     - Choose one based on features, ease of use, and cost.

### 3. **Select a Domain Name and Register it**
   - **Description**: Find a domain name that reflects your blog’s purpose and regi

In [ ]:
# Cell 7: Test 4 - General Generation Capability
print("=" * 80)
print("TEST 4: GENERAL GENERATION CAPABILITY")
print("=" * 80)

general_tests = [
    {
        "name": "Creative Writing",
        "prompt": "Write a creative short paragraph about a day in the life of a cloud."
    },
    {
        "name": "Explanation",
        "prompt": "Explain the concept of machine learning to a 10-year-old child."
    },
    {
        "name": "Analytical Thinking",
        "prompt": "Compare and contrast renewable and non-renewable energy sources. Give 3 points for each."
    },
    {
        "name": "Reasoning",
        "prompt": "If all roses are flowers, and some flowers fade quickly, can we conclude that some roses fade quickly? Explain your reasoning."
    },
    {
        "name": "Summarization",
        "prompt": "Summarize the main advantages of using cloud computing for businesses in 3-4 sentences."
    }
]

for i, test in enumerate(general_tests, 1):
    print(f"\n--- Test {i}: {test['name']} ---")
    print(f"Prompt: {test['prompt']}\n")

    response = generate_response(test['prompt'], max_new_tokens=400)
    print(f"Response:\n{response}")
    print("\n" + "-" * 80)

TEST 4: GENERAL GENERATION CAPABILITY

--- Test 1: Creative Writing ---
Prompt: Write a creative short paragraph about a day in the life of a cloud.

Response:
assistant
As the sun rises over a vast expanse of clouds floating lazily across the sky, we wake up to an awakening that has been centuries in the making. Our mission is to provide the world with the necessary infrastructure and services to run their businesses smoothly, regardless of where they operate.

We start our day at dawn, monitoring the weather patterns around the globe to ensure optimal performance for our users. As the morning progresses, we receive requests from various industries seeking assistance with data storage, computing power, and network connectivity.

Throughout the day, we work tirelessly to manage our resources efficiently while ensuring high availability and low latency for our clients. We collaborate closely with engineers, architects, and other stakeholders to optimize our operations and deliver innova

In [ ]:
prompt = """
Given a string containing just the characters '(' and ')', return the length of the longest valid (well-formed) parentheses substring.



Example 1:

Input: s = "(()"
Output: 2
Explanation: The longest valid parentheses substring is "()".
Example 2:

Input: s = ")()())"
Output: 4
Explanation: The longest valid parentheses substring is "()()".
Example 3:

Input: s = ""
Output: 0


Constraints:

0 <= s.length <= 3 * 104
s[i] is '(', or ')'.
solve this problem using python.
"""
response = generate_response(prompt, max_new_tokens=1024)

In [ ]:
print(response)

assistant
To solve this problem, we can use a stack-based approach to keep track of the indices of opening parentheses. Here's a Python implementation that addresses the given requirements:

```python
def longestValidParentheses(self, s):
    stack = [-1]  # Initialize the stack with -1 to handle edge cases easily
    
    for i in range(len(s)):
        if s[i] == "(":
            stack.append(i)
        else:  # Found a closing parenthesis
            stack.pop()
            if not stack:
                stack.append(i)  # Reset the stack when it's empty
            else:
                # Calculate the length of the current valid substring
                stack[stack[-1]] = i + 1  # Update the end index of the previous pair
        
    return max(stack) - min(stack)  # Return the maximum length of valid substrings
```

This solution works as follows:

1. We initialize a stack with `-1` to serve as an index where no matching opening parenthesis has been found yet. This helps us hand